<img src="https://prnewswire2-a.akamaihd.net/p/1893751/sp/189375100/thumbnail/entry_id/1_su9da4fu/def_height/1001/def_width/1911/version/100011/type/2/q/100"  width="300" height="200">

# Data Acquisition

We need to acquire data to meet our client's request(s):

>Use the 2017 properties and predictions data for <strong>single unit / single family homes</strong>... [include] the discoveries you made and work you have done related to <strong> uncovering the drivers Zillow's Zestimate error. 


# Define a single unit / single family home
This definition comes from the [US Census](https://www.census.gov/construction/chars/definitions/) website.

   
<strong>Attached and Detached Single-Family Housing Units</strong>
>
> Single-family structures include fully detached, semi-detached (semi-attached, side-by-side), row houses, duplexes, quadruplexes, and townhouses. In order for attached units to be classified as single-family structures, each unit must:
>
> - Be separated by a ground-to roof wall,
> - Have a separate heating system,
> - Have individual meters for public utilities, and
> - Have no units located above or below.
>
> If each unit within the building does not meet the conditions above, the building is considered multifamily.


Definition from The City of Los Angeles Municipal [codes](https://codelibrary.amlegal.com/codes/los_angeles/latest/lapz/0-0-0-886).
    
    
<strong>DWELLING UNIT</strong>
>     
> A group of two or more rooms, one of which is a kitchen, designed for occupancy by one family for living and sleeping purposes.  (Amended by Ord. No. 107,884, Eff. 9/23/56.)    
    
<strong>FAMILY</strong>
>
> One or more persons living together in a dwelling unit, with common access to, and common use of all living, kitchen, and eating areas within the dwelling unit.  (Amended by Ord. No. 177,325, Eff. 3/18/06.)
    
<br>
So now that know what properties we're looking for, let's create a map so we know where we're going. 
    
## Data Acquisition Workflow
Step 1: SQL Query<br>
Step 2: Local cached file


# Step 1: SQL Query
---
1. Database structure
1. SQL Query
1. Load data from the database to local environment

## 1. Database Structure
Before we import the data to our local environment, we need to ask ourselves 3 _important_ questions.

1. What data do I need? / What data does my team need?

> I need data describing properties and predictions for single unit homes sold in 2017. I need this data to discover the drivers of logerror. It's crucial to know this step. Reminder to self:
> 
> `"What's the business problem I'm trying to solve for my client."`
>
> Acquiring irrelevant data wastes time and reveals irrelevant insights.

2. Where is the data stored and what form(s) is it in?

> Our Zillow data is stored on a SQL database under the database name: `zillow`. Relevant data to the project is spread across several tables (listed below).

3. Before I begin aquiring the data from the database, can I explain why I selected these tables?

> In addition to `properties_2017` and `prediction_2017`, several `{key: value}` tables are needed in our query. These table will act like a rosetta stone for our main table, `properties_2017`. In `properties_2017`, categorical values are stored as numeric values. We have no clue what the numbers represent without context. The `{key: value}` tables describe what each unique numeric value represents. <strong>We can use the actual `{key: value} names` during exploration to discover the drivers of Zillow's Zestimate logerror</strong>.

<br>

SQL Database Name: `zillow`

| SQL Table Name | Contents | Features |
| :------------- | :------- | :------- |
| `airconditioningtype` | Indicates the properties air conditioning type | `airconditioningtypeid`, `airconditioningdesc`|
| `architecturalstyletype` | Indicates the architectural style of the property | `architecturalstyletypeid`, `architecturalstyletype` |
| `buildingclasstype` | Indicates the type of fireproofing a property has | `buildingclasstypeid`, `buildingclassdesc` |
| `heatingorsystemtype` | Indicates how the property is heated | `heatingorsystemtypeid`, `heatingorsystemdesc` |
| `predictions_2017` | The last date a property was sold with a corresponding Zestimate logerror for 2017 | `parcelid`, `logerror`, `transactiondate` |
| `propertylandusetype` | The type of property that sits on a lot | `propertylandusetypeid`, `propertylandusedesc` |
| `storytype` | Indicates the number of stories a property has | `storytypeid`, `storydesc` |
| `typeconstructiontype` | Indicates the type of material used to construct the property | `typeconstructiontypeid`, `typeconstructiondesc` |
| `properties_2017` | Properties that were sold in 2017. Each property has unique id with features describing the property | `parcelid`,`id`,`basementsqft`,`bathroomcnt`, `bedroomcnt`,`buildingqualitytypeid`,`calculatedbathnbr`, `decktypeid`,`finishedfloor1squarefeet`,`calculatedfinishedsquarefeet`, `finishedsquarefeet12`,`finishedsquarefeet13`,`finishedsquarefeet15`, `finishedsquarefeet50`,`finishedsquarefeet6`, `fips`,`fireplacecnt`,`fullbathcnt`,`garagecarcnt`, `garagetotalsqft`,`hashottuborspa`,`latitude`,`longitude`, `lotsizesquarefeet`,`poolcnt`,`poolsizesum`,`pooltypeid10`, `pooltypeid2`,`pooltypeid7`,`propertycountylandusecode`, `propertyzoningdesc`,`rawcensustractandblock`, `regionidcity`,`regionidcounty`,`regionidneighborhood`, `regionidzip`,`roomcnt`,`threequarterbathnbr`,`unitcnt`, `yardbuildingsqft17`,`yardbuildingsqft26`,`yearbuilt`, `numberofstories`,`fireplaceflag`,`structuretaxvaluedollarcnt`, `taxvaluedollarcnt`,`assessmentyear`, `landtaxvaluedollarcnt`,`taxamount`,`taxdelinquencyflag`, `taxdelinquencyyear`,`censustractandblock`|

<br>
<br>

## 2. SQL Query
Let's take a look at our main table: `properties_2017`. Using <strong><font color='blue'>select * from properties_2017;</font></strong>, I noticed many columns contained missing values.

Because of this, tables that are joined onto `properties_2017` need to be joined using: <strong><font color='blue'> left join </font></strong>

<img src="https://images.squarespace-cdn.com/content/v1/5732253c8a65e244fd589e4c/1464122797709-C2CDMVSK7P4V0FNNX60B/ke17ZwdGBToddI8pDm48kMjn7pTzw5xRQ4HUMBCurC5Zw-zPPgdn4jUwVcJE1ZvWEV3Z0iVQKU6nVSfbxuXl2c1HrCktJw7NiLqI-m1RSK4p2ryTI0HqTOaaUohrI8PIO5TUUNB3eG_Kh3ocGD53-KZS67ndDu8zKC7HnauYqqk/image-asset.png"  width="600" height="500">

If the left table has a missing value at a certain observation, a left join preserves the original shape/columns/rows of the left table by inserting columns from the right table as a <strong><font color='blue'>NULL</font></strong> value.

> We want as much information from our data source that:
> 1. Is within the scope of the project/ relevant to the business problem.
> 2. Helps us to explain insights/ drivers of Zillow's Zestimate logerror.

Now that we know which tables to query and how to combine them, let's write the SQL query to retreive the data.

<br>

```sql
select *
from properties_2017
join(select parcelid,
    logerror,
    max(transactiondate) as lasttransactiondate
    from predictions_2017
    group by parcelid, logerror
    ) as predictions using(parcelid)
left join `airconditioningtype` using(`airconditioningtypeid`)
left join `architecturalstyletype` using(`architecturalstyletypeid`)
left join `buildingclasstype` using(`buildingclasstypeid`)
left join `heatingorsystemtype` using(`heatingorsystemtypeid`)
left join `propertylandusetype` using(`propertylandusetypeid`)
left join `storytype` using(`storytypeid`)
left join `typeconstructiontype` using(`typeconstructiontypeid`)
where (latitude is not null
and longitude is not null
and propertylandusetypeid in (261, 262, 264, 273));
```
You can find a break down of the SQL query in the [appendix](#Appendix).

# 3. Load data from the database to our local environment

In [5]:
import pandas as pd
import numpy as np

import env
from acquire import get_zillow_data
from warnings import filterwarnings
from IPython.display import Video

filterwarnings('ignore')

In [2]:
df = get_zillow_data()

```python
sql_query ='''
select *
from properties_2017
join(select parcelid,
    logerror,
    max(transactiondate) as lasttransactiondate
    from predictions_2017
    group by parcelid, logerror
    ) as predictions using(parcelid)
left join `airconditioningtype` using(`airconditioningtypeid`)
left join `architecturalstyletype` using(`architecturalstyletypeid`)
left join `buildingclasstype` using(`buildingclasstypeid`)
left join `heatingorsystemtype` using(`heatingorsystemtypeid`)
left join `propertylandusetype` using(`propertylandusetypeid`)
left join `storytype` using(`storytypeid`)
left join `typeconstructiontype` using(`typeconstructiontypeid`)
where (latitude is not null
and longitude is not null
and propertylandusetypeid in (261, 262, 264, 273));'''

file = 'zillow.csv'

if os.path.isfile(file):
    return pd.read_csv('zillow.csv')
else:
    df = pd.read_sql(sql_query, get_connection('zillow'))
    df.to_csv('zillow.csv', index=False)
    return df
```

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52445 entries, 0 to 52444
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        76 non-null     float64
 1   storytypeid                   47 non-null     float64
 2   propertylandusetypeid         52445 non-null  float64
 3   heatingorsystemtypeid         33941 non-null  float64
 4   buildingclasstypeid           0 non-null      object 
 5   architecturalstyletypeid      70 non-null     float64
 6   airconditioningtypeid         13644 non-null  float64
 7   parcelid                      52445 non-null  int64  
 8   id                            52445 non-null  int64  
 9   basementsqft                  47 non-null     float64
 10  bathroomcnt                   52445 non-null  float64
 11  bedroomcnt                    52445 non-null  float64
 12  buildingqualitytypeid         33746 non-null  float64
 13  c

In [12]:
df.propertylandusedesc.value_counts()

Single Family Residential    52439
Townhouse                        6
Name: propertylandusedesc, dtype: int64

# Appendix

Let's break down the query
```sql
-- First two lines establish the primary table to join on.
select *
from properties_2017

-- This subquery selects parcelid, logerror, and the last transaction date a property was sold in 2017. If a property was sold multiple times, we want the most recent date the property was sold. If a property was sold only one time, that's okay too. max(transactiondate) combined with a group by statement will retrieve the most recent transaction dates for a given parcelid. We can join this data on `properties_2017` using the `parcelid` column. 
join(select parcelid,
    logerror,
    max(transactiondate) as lasttransactiondate
    from predictions_2017
    group by parcelid, logerror
    ) as predictions using(parcelid)


-- As discussed above using the left/right join diagram, we'll join the rest of the tables using a left join to preserve the columns/rows in `properties_2017`
left join `airconditioningtype` using(`airconditioningtypeid`)
left join `architecturalstyletype` using(`architecturalstyletypeid`)
left join `buildingclasstype` using(`buildingclasstypeid`)
left join `heatingorsystemtype` using(`heatingorsystemtypeid`)
left join `propertylandusetype` using(`propertylandusetypeid`)
left join `storytype` using(`storytypeid`)
left join `typeconstructiontype` using(`typeconstructiontypeid`)

-- Use a where clause to filter for properties that have non-null latitude and longitude values.
where (latitude is not null
and longitude is not null
-- Filter for properties labeled as single unit / single family homes
and propertylandusetypeid in (261, 262, 264, 273));
-- 261 == Single Family Residential
-- 262 == Rural Residence
-- 264 == Townhouse
-- 273 == Bungalow

-- Include after MVP
-- Filter for single story properties
-- 246 == Duplex (2 units any combination)
-- 247 == Triplex
-- 248 == Quadruplex
```

[Return to Loading data from the database](#3._Load_data_from_the_database_to_our_local_environment)